In [1]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from helper_functions import lr_schedule
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam


print(tf.__version__)
BATCH_SIZE = 32
IMAGE_DIMS = (256, 256, 3)
from tensorflow.keras.callbacks import Callback
import numpy as np

def lr_schedule(epoch):
    initial_lr = 0.001
    drop = 0.5
    epoch_drop = 2.0
    lr = initial_lr * (drop ** (epoch // epoch_drop))
    print(f"Epoch {epoch}: Learning rate = {lr}")
    return lr


2024-12-24 04:43:29.281215: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-24 04:43:35.891332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1


In [2]:
train_dir = r'/home/arsenal/ml/OCT2017/train'
val_dir = r'/home/arsenal/ml/OCT2017/val'

train_dataset = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256,256),
    batch_size=BATCH_SIZE
)

val_dataset = keras.preprocessing.image_dataset_from_directory(
    val_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256,256),
    batch_size=BATCH_SIZE
)

class_names = train_dataset.class_names
print("Class names:", class_names)

Found 83489 files belonging to 4 classes.
Using 66792 files for training.


2024-12-24 04:43:34.183646: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-24 04:43:34.206757: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-24 04:43:34.206809: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-24 04:43:34.211808: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-24 04:43:34.211993: I external/local_xla/xla/stream_executor

Found 32 files belonging to 4 classes.
Using 6 files for validation.
Class names: ['CNV', 'DME', 'DRUSEN', 'NORMAL']


In [3]:
import tensorboard

model1 = keras.models.Sequential([
    keras.layers.Input(shape=IMAGE_DIMS),
    keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model1.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['accuracy'])

model2 = keras.models.Sequential([
    keras.layers.Input(shape=IMAGE_DIMS),
    keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model2.compile(optimizer=Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

input = keras.layers.Input(shape=IMAGE_DIMS)

model1_output = model1(input)
model2_output = model2(input)

combined_output = keras.layers.Concatenate()([model1_output, model2_output])
final_model = keras.Model(inputs=input, outputs=combined_output)

tensorboard_callback = keras.callbacks.TensorBoard('assets/logs/multiModel')

final_model.compile(optimizer=Adam(),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

final_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 1)         │ 31,509,697 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 3)         │ 31,509,955 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 4)         │          0 │ sequential[0][0], │
│ (Concatenate)       │                   │            │ sequential_1[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 63,019,652 (240.40 MB)

 Trainable params: 63,019,652 (240.40 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
def one_hot_encode(dataset, num_classes):
    def encode(image, label):
        label = tf.one_hot(label, num_classes)
        return image, label
    return dataset.map(encode)

num_classes = 4

train_dataset_encoded = one_hot_encode(train_dataset, num_classes)
val_dataset_encoded = one_hot_encode(val_dataset, num_classes)
lr_scheduler = LearningRateScheduler(lr_schedule)

if not final_model.built:
    final_model.compile(optimizer=Adam(),
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

final_model.build(input_shape=(None, *IMAGE_DIMS))

history = final_model.fit(train_dataset_encoded,
                          epochs=15,
                          validation_data=val_dataset_encoded,
                          batch_size = 32,
                          callbacks=[lr_scheduler,tensorboard_callback])


Epoch 0: Learning rate = 0.001
Epoch 1/15


I0000 00:00:1735015416.517529    7195 service.cc:145] XLA service 0x7f76c400fdf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735015416.517610    7195 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2024-12-24 04:43:36.579126: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-24 04:43:36.858850: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-12-24 04:43:45.838004: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng56{k2=0,k13=2,k14=3} for conv (f32[32,32,127,127]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,125,125]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait

2041/2088 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.4448 - loss: 4.3550

2024-12-24 04:47:01.048765: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[8,64,125,125]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,32,127,127]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-12-24 04:47:01.081878: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 6.589869999s
Trying algorithm eng0{} for conv (f32[8,64,125,125]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,32,127,127]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],

2088/2088 ━━━━━━━━━━━━━━━━━━━━ 201s 93ms/step - accuracy: 0.4448 - loss: 4.3550 - val_accuracy: 0.3333 - val_loss: 3.2640 - learning_rate: 0.0010
Epoch 1: Learning rate = 0.001
Epoch 2/15
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 191s 92ms/step - accuracy: 0.4459 - loss: 4.3248 - val_accuracy: 0.3333 - val_loss: 3.2640 - learning_rate: 0.0010
Epoch 2: Learning rate = 0.0005
Epoch 3/15
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 188s 90ms/step - accuracy: 0.4461 - loss: 4.3206 - val_accuracy: 0.3333 - val_loss: 3.2640 - learning_rate: 5.0000e-04
Epoch 3: Learning rate = 0.0005
Epoch 4/15
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 189s 91ms/step - accuracy: 0.4462 - loss: 4.3237 - val_accuracy: 0.3333 - val_loss: 3.2640 - learning_rate: 5.0000e-04
Epoch 4: Learning rate = 0.00025
Epoch 5/15
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 193s 92ms/step - accuracy: 0.4464 - loss: 4.3165 - val_accuracy: 0.3333 - val_loss: 3.2640 - learning_rate: 2.5000e-04
Epoch 5: Learning rate = 0.00025
Epoch 6/15
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 193s 92ms/s

In [5]:
%pip install tensorboard

Note: you may need to restart the kernel to use updated packages.


In [6]:
%load_ext tensorboard

In [7]:
%tensorboard --logdir=assets/logs/multiModel

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Step 2: Load the image you want to predict
img_path = '/home/arsenal/ml/DRUSEN-9837663-1.jpeg'  # Update with your image path

# Load the image and resize it to the target size used during training (256, 256)
img = image.load_img(img_path, target_size=(256, 256), color_mode='grayscale')

# Step 3: Preprocess the image
# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Normalize the image (if your training data was normalized by dividing by 255)
img_array = img_array / 255.0  # Scale pixel values to [0, 1]

# Reshape the image to match the input shape the model expects (add batch dimension)
img_array = np.expand_dims(img_array, axis=0)  # Shape will be (1, 256, 256, 1)

# Step 4: Make predictions
predictions = model.predict(img_array)

# Step 5: Process the predictions
# For multi-class classification, predictions will be a probability distribution over all classes
predicted_class = np.argmax(predictions, axis=1)[0]  # Get the index of the highest probability class

# Step 6: Map the predicted class to class names
class_names = ['Class1', 'Class2', 'Class3', 'Class4']  # Replace with your actual class names
predicted_class_name = class_names[predicted_class]

# Step 7: Display the result
print(f'Predicted class index: {predicted_class}')
print(f'Predicted class name: {predicted_class_name}')

# Optional: Display the image along with the prediction
plt.imshow(img, cmap='gray')
plt.title(f'Predicted Class: {predicted_class_name}')
plt.axis('off')  # Hide axes for clarity
plt.show()
